## Ensure a Minikube cluster is running

- If you created a Minikube cluster with the command `minikube start --nodes 3` and it is still running:
  - Do nothing
- If you have a stopped (not deleted) Minikube cluster that was created with the command `minikube start --nodes 3`:
  - Run the command `minikube start`
- If you don't have a Minikube cluster (or deleted your previous one):
  - Run the command `minikube start --nodes 3`  

## List Nodes

- The command `kubectl get nodes` is used to list the nodes running in a kubernetes cluster.
  - Without the `-o wide` option, provides basic information about the nodes.
  - With The `-o wide` option, provides additional information about the nodes.
- A Minikube cluster always has 1 `control-plane` node (called `minikube` here).
- Additional nodes will be `worker` nodes (called `minikube-02` and `minikube-03` here).
- Here we see:
  - The `NAME` of each node.
  - The `ROLES` of each node (`minikube` is a Control Plane node, whereas `minikube-m02` and `minikube-m03` are Worker nodes).
  - The `VERSION` or kubernetes each node is using.
  - The `INTERNAL-IP` addess of each node (the IP address each node has within the kubernetes cluster).
  - The `EXTERNAL-IP` addess of each node (the public IP address each node has outside the kubernetes cluster).
  - The `OS-IMAGE` used by each node (in this case Ubuntu 22.04).

In [1]:
#!kubectl get nodes
!kubectl get nodes -o wide

NAME           STATUS   ROLES           AGE   VERSION   INTERNAL-IP    EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION                       CONTAINER-RUNTIME
minikube       Ready    control-plane   79s   v1.28.3   192.168.49.2   <none>        Ubuntu 22.04.3 LTS   5.15.133.1-microsoft-standard-WSL2   docker://24.0.7
minikube-m02   Ready    <none>          63s   v1.28.3   192.168.49.3   <none>        Ubuntu 22.04.3 LTS   5.15.133.1-microsoft-standard-WSL2   docker://24.0.7
minikube-m03   Ready    <none>          48s   v1.28.3   192.168.49.4   <none>        Ubuntu 22.04.3 LTS   5.15.133.1-microsoft-standard-WSL2   docker://24.0.7


## Get information about a node

- The command `kubectl get node [NodeName]` returns information about a node with name `[NodeName]`.
  - The `-o yaml` option returns the information as a YAML manifest (definition) for a node.

- Some important information contained in a node listing:
  ```bash
  apiVersion: v1
  kind: Node
  metadata:
    name: minikube           <--- the name of the node
  spec:
    podCIDR: 10.244.0.0/24   <--- the address range for Pods allocated on this node
  status:
    addresses:
    - address: 192.168.49.2  <--- the node's IP addres
      type: InternalIP
    - address: minikube      <--- the node's hostname
      type: Hostname
    capacity:
      cpu: 16                          <--- number of CPU threads (number of CPU cores => divide by 2)
      ephemeral-storage: 1055762868Ki  <--- storage (disk size) in kibibytes.
      memory: 16279876Ki               <--- RAM in kibibytes
      pods: 110                        <--- maximum number of Pods that the ndoe can run
    images <list>                      <--- list of container images downloaded to the node
    nodeInfo:
      architecture: amd64
      containerRuntimeVersion: docker://24.0.7           <--- container runtime version
      kernelVersion: 5.15.133.1-microsoft-standard-WSL2
      kubeProxyVersion: v1.28.3                          <--- kube proxy version
      kubeletVersion: v1.28.3                            <--- kubelet version
      operatingSystem: linux                             <--- operating system type
      osImage: Ubuntu 22.04.3 LTS
  ```

In [2]:
#!kubectl get node minikube
!kubectl get node minikube -o yaml

apiVersion: v1
kind: Node
metadata:
  annotations:
    kubeadm.alpha.kubernetes.io/cri-socket: unix:///var/run/cri-dockerd.sock
    node.alpha.kubernetes.io/ttl: "0"
    volumes.kubernetes.io/controller-managed-attach-detach: "true"
  creationTimestamp: "2024-02-07T19:27:27Z"
  labels:
    beta.kubernetes.io/arch: amd64
    beta.kubernetes.io/os: linux
    kubernetes.io/arch: amd64
    kubernetes.io/hostname: minikube
    kubernetes.io/os: linux
    minikube.k8s.io/commit: 8220a6eb95f0a4d75f7f2d7b14cef975f050512d
    minikube.k8s.io/name: minikube
    minikube.k8s.io/primary: "true"
    minikube.k8s.io/updated_at: 2024_02_07T20_27_31_0700
    minikube.k8s.io/version: v1.32.0
    node-role.kubernetes.io/control-plane: ""
    node.kubernetes.io/exclude-from-external-load-balancers: ""
  name: minikube
  resourceVersion: "541"
  uid: e451cadc-d74b-4613-a8e9-9fca411b1967
spec:
  podCIDR: 10.244.0.0/24
  podCIDRs:
  - 10.244.0.0/24
status:
  addresses:
  - address: 192.168.49.2
    type: In

## Describe a node

- The command `kubectl describe nodes` returns detailed information about all nodes.
- The command `kubectl describe node [NodeName]` returns detailed information about a node with name `[NodeName]`.
- A node description contains the same, but more detailed, information as a node YAML listing above.
- In addition, a node description contains a list of important `Events` that have taken place (at the bottom of the listing), e.g.:
 
  ```bash
  Events:
    Type    Reason                   Age                From             Message
    ----    ------                   ----               ----             -------
    Normal  Starting                 50m                kube-proxy       
    Normal  Starting                 51m                kubelet          Starting kubelet.
  ```

In [3]:
#!kubectl describe nodes
!kubectl describe node minikube
#!kubectl describe node minikube-m02
#!kubectl describe node minikube-m03

Name:               minikube
Roles:              control-plane
Labels:             beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/os=linux
                    kubernetes.io/arch=amd64
                    kubernetes.io/hostname=minikube
                    kubernetes.io/os=linux
                    minikube.k8s.io/commit=8220a6eb95f0a4d75f7f2d7b14cef975f050512d
                    minikube.k8s.io/name=minikube
                    minikube.k8s.io/primary=true
                    minikube.k8s.io/updated_at=2024_02_07T20_27_31_0700
                    minikube.k8s.io/version=v1.32.0
                    node-role.kubernetes.io/control-plane=
                    node.kubernetes.io/exclude-from-external-load-balancers=
Annotations:        kubeadm.alpha.kubernetes.io/cri-socket: unix:///var/run/cri-dockerd.sock
                    node.alpha.kubernetes.io/ttl: 0
                    volumes.kubernetes.io/controller-managed-attach-detach: true
CreationTimestamp:  Wed, 07 Fe